In [1]:
import threading

In [2]:
def task():
    print("hello from task, thread:", threading.get_native_id())

t = threading.Thread(target=task)
t.start()
print("hello from main, thread:", threading.get_native_id())

hello from task, thread:hello from main, thread: 1405
 1418


In [3]:
total = 0
def task2(count):
    global total
    for i in range(count):
        total += 1

t = threading.Thread(target=task2, args=[1_000_000])
t.start()
t.join() # wait for the thread to finish
print(total)

1000000


In [4]:
total = 0
def task3(count):
    global total
    for i in range(count):
        total += 1

t1 = threading.Thread(target=task3, args=[1_000_000])
t2 = threading.Thread(target=task3, args=[1_000_000])
t1.start()
t2.start()
t1.join() # wait for the thread to finish
t2.join()
print(total)

1158077


In [5]:
import dis

In [6]:
print(dis.dis("total += 1"))

  0           RESUME                   0

  1           LOAD_NAME                0 (total)
              LOAD_CONST               0 (1)
              BINARY_OP               13 (+=)
              STORE_NAME               0 (total)
              RETURN_CONST             1 (None)
None


# Locks

In [13]:
%%time
# no locks: 205 ms
# with lock (fine grained): 463 ms
# with lock (coarse grained): 154 ms

lock = threading.Lock() # protects total
total = 0

def task4(count):
    global total
    lock.acquire()
    for i in range(count):
        total += 1
    lock.release()

t1 = threading.Thread(target=task4, args=[1_000_000])
t2 = threading.Thread(target=task4, args=[1_000_000])
t1.start()
t2.start()
t1.join() # wait for the thread to finish
t2.join()
print(total)

2000000
CPU times: user 155 ms, sys: 0 ns, total: 155 ms
Wall time: 154 ms


In [2]:
import threading

In [11]:
bank_accounts = {"x": 25, "y": 100, "z": 200} # in dollars
lock = threading.Lock() # protects bank_accounts

def transfer(src, dst, amount):
    with lock:  # acquire() for me
        success = False
        if bank_accounts[src] >= amount:
            bank_accounts[src] -= amount
            bank_accounts[dst] += amount
            success = True
        print("transferred" if success else "denied")
    # lock.release() # xhappens automatically

In [10]:
transfer("x", "y", 20)
bank_accounts

transferred


RuntimeError: release unlocked lock

In [5]:
transfer("x", "y", 20)
bank_accounts

denied


{'x': 5, 'y': 120, 'z': 200}

In [6]:
transfer("w", "y", 20) # there is no w account!
bank_accounts

KeyError: 'w'

In [7]:
transfer("x", "y", 1)
bank_accounts

transferred


{'x': 4, 'y': 121, 'z': 200}

In [ ]:
# always ask: if there is an error, do we still release resources?  (locks, memory, files, etc)